### Importando as bibliotecas

In [1]:
import os
import sys
sys.path.append('C:/Users/fabri/Documents/GitHub/MovieRecommendation')
os.environ['DJANGO_SETTINGS_MODULE'] = 'MovieRecomendation.settings'

In [2]:
import django
django.setup()

In [3]:
#fazendo os imports necessarios
from recommendations.utils import get_all_movies, search_movies

In [4]:
movies = get_all_movies()

In [5]:
print(movies)

{'page': 1, 'results': [{'adult': False, 'backdrop_path': '/4woSOUD0equAYzvwhWBHIJDCM88.jpg', 'genre_ids': [28, 27, 53], 'id': 1096197, 'original_language': 'en', 'original_title': 'No Way Up', 'overview': "Characters from different backgrounds are thrown together when the plane they're travelling on crashes into the Pacific Ocean. A nightmare fight for survival ensues with the air supply running out and dangers creeping in from all sides.", 'popularity': 2991.229, 'poster_path': '/hu40Uxp9WtpL34jv3zyWLb5zEVY.jpg', 'release_date': '2024-01-18', 'title': 'No Way Up', 'video': False, 'vote_average': 5.921, 'vote_count': 101}, {'adult': False, 'backdrop_path': '/xOMo8BRK7PfcJv9JCnx7s5hj0PX.jpg', 'genre_ids': [28, 12, 878], 'id': 693134, 'original_language': 'en', 'original_title': 'Dune: Part Two', 'overview': 'Follow the mythic journey of Paul Atreides as he unites with Chani and the Fremen while on a path of revenge against the conspirators who destroyed his family. Facing a choice betw

In [6]:
query = 'Inception'
search_results = search_movies(query)

In [7]:
print(search_results)

{'page': 1, 'results': [{'adult': False, 'backdrop_path': '/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg', 'genre_ids': [28, 878, 12], 'id': 27205, 'original_language': 'en', 'original_title': 'Inception', 'overview': 'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.', 'popularity': 111.555, 'poster_path': '/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg', 'release_date': '2010-07-15', 'title': 'Inception', 'video': False, 'vote_average': 8.367, 'vote_count': 35343}, {'adult': False, 'backdrop_path': '/JeGkRdNsOuMrgwBdtB0hp763MU.jpg', 'genre_ids': [18, 53], 'id': 613092, 'original_language': 'es', 'original_title': 'El crack cero', 'overview': 'Madrid, Spain, 1975; shortly after the end of the Franco dictatorship. Six months after the mysterious death of his lover, a prestigious tail